In [1]:
import pandas as pd
import numpy as np


In [2]:
df1= pd.read_excel('Dataset.xlsx') 
df=df1.sample(frac = 0.75)

In [3]:
df.head()

,SL,Emotion,Text,Year,Racial Biases,Hatred Percentage,Depression Percentage
15524,15525,angry,ভালবাসতে জানেন পাইতে ও নিতে পারেন কিন্তু দিতে ...,2018,NaN,NaN,NaN
35245,35246,fear,থাকতে হবে সতর্ক ও সচেতন,2021,NaN,NaN,NaN
17575,17576,surprise,মামা বাড়ীর আবদার!,2018,NaN,NaN,NaN
19909,19910,disgust,"আমারা যারা ছাএ,তারা কি স্টক করে মারা যাবো নাকি",2019,NaN,NaN,NaN
25370,25371,disgust,ভাই পাগলের সাথে কি কন!! বাদদেন!! বাংলাদেশে একট...,2020,NaN,NaN,NaN


In [4]:
df.tail()

,SL,Emotion,Text,Year,Racial Biases,Hatred Percentage,Depression Percentage
11114,11115,happy,দেশটাকে শেখ হাসিনার সরকার উন্নয়নের শিখরে পৌঁছে...,2017,NaN,NaN,NaN
28148,28149,sad,"ভারতে করোনায় আবারও রেকর্ড সংক্রমণ, 'রেকর্ড' মৃ...",2020,NaN,NaN,NaN
23171,23172,surprise,হঠাৎ ব্রাহ্মণবাড়িয়া প্রেসক্লাবে হেফাজতের নেতারা,2019,NaN,NaN,NaN
23112,23113,surprise,প্রথমে এলিজাবেথই প্রিন্স ফিলিপের প্রেমে পড়েন।,2019,NaN,NaN,NaN
16507,16508,disgust,তাঁর সংসদীয় এলাকায় এক সাংবাদিককে তাঁর লোকেরা হ...,2018,NaN,NaN,NaN


In [5]:
df["Emotion"].value_counts()

disgust     4548
sad         4524
fear        4503
happy       4490
surprise    4475
angry       4460
Name: Emotion, dtype: int64

In [6]:
class_names = ['disgust','angry','happy','surprise','fear','sad']

In [7]:
encoding = {
    'disgust': 0, 
    'angry' :1,
    'happy':2,
    'surprise':3,
    'fear':4,
    'sad':5
}

In [8]:
text = df['Text'].values.tolist()
emotion= df['Emotion'].tolist()

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text , emotion, test_size=.2, random_state=42)

In [10]:
df["Emotion"].value_counts()

disgust     4548
sad         4524
fear        4503
happy       4490
surprise    4475
angry       4460
Name: Emotion, dtype: int64

In [11]:
import ktrain
from ktrain import text
MODEL_NAME = 'sagorsarker/bangla-bert-base'
t = text.Transformer(MODEL_NAME, maxlen=500)


In [12]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 16
	95percentile : 44
	99percentile : 69


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 16
	95percentile : 44
	99percentile : 67


In [13]:
model = t.get_classifier()

In [14]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [15]:
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
3600/3600 [==============================] - 38860s 11s/step - loss: 1.4698 - accuracy: 0.4169 - val_loss: 1.0206 - val_accuracy: 0.6500
Epoch 2/5
3600/3600 [==============================] - 39394s 11s/step - loss: 0.8085 - accuracy: 0.7254 - val_loss: 0.7252 - val_accuracy: 0.7609
Epoch 3/5
3600/3600 [==============================] - 40366s 11s/step - loss: 0.4975 - accuracy: 0.8378 - val_loss: 0.5908 - val_accuracy: 0.8180
Epoch 4/5
3600/3600 [==============================] - 39903s 11s/step - loss: 0.2601 - accuracy: 0.9141 - val_loss: 0.5850 - val_accuracy: 0.8363
Epoch 5/5
3600/3600 [==============================] - 40008s 11s/step - loss: 0.1311 - accuracy: 0.9536 - val_loss: 0.5989 - val_accuracy: 0.8469


In [16]:
learner.validate(val_data=(val), class_names=class_names)

              precision    recall  f1-score   support

     disgust       0.84      0.84      0.84       884
       angry       0.85      0.89      0.87       954
       happy       0.87      0.91      0.89       850
    surprise       0.88      0.86      0.87       903
        fear       0.78      0.70      0.74       897
         sad       0.84      0.89      0.86       912

    accuracy                           0.85      5400
   macro avg       0.85      0.85      0.85      5400
weighted avg       0.85      0.85      0.85      5400



array([[739,  47,  12,  17,  40,  29],
       [ 27, 849,   9,  21,  30,  18],
       [ 15,   8, 772,   6,  33,  16],
       [ 20,  11,  24, 773,  45,  30],
       [ 57,  62,  49,  40, 630,  59],
       [ 17,  20,  19,  18,  28, 810]], dtype=int64)